In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler

In [15]:
df = pd.read_csv(r'D:\tesis_u_sabana\data\db_0\db_0.csv', sep="~", encoding='utf-8')

In [16]:
df['nit_entidad_x']=df['nit_entidad_x'].astype(str)
df['codigo_proveedor']=df['codigo_proveedor'].astype(str)
df['sin_modificaciones'] = df['sin_modificaciones'].astype(bool)
df = df.rename(columns={'duracion_en_dias_y': 'duracion_proceso_dias'})

In [17]:
df.head()

id_contrato                                     nombre_entidad  \
0  co1.pccntr.1358665  sena regional antioquia grupo administrativo c...   
1  co1.pccntr.4243635                              departamento del meta   
2  co1.pccntr.4381704  empresa de servicios publicos de acueducto; al...   
3  co1.pccntr.5334124                                                unp   
4  co1.pccntr.2164831  sena regional norte de santander grupo de apoy...   

  nit_entidad_x                                objeto_del_contrato  \
0     899999034  prestacion de los servicios personales de cara...   
1    8920001488  fortalecer el departamento administrativo de p...   
2     901142488  prestacion de servicios de apoyo a la gestion ...   
3     900475780  prestar los servicios de apoyo a la gestion en...   
4     899999034  54_9119_473 prestar servicios personales de ca...   

                          objeto_del_contrato_limpio  \
0  ['personales', 'caracter', 'temporal', 'interp...   
1  ['fortalecer', 'departamento', 'administrativo...   
2  ['empresa', 'publicos', 'realizacion', 'tratam...   
3  ['misionales', 'desarrolla', 'subdireccion', '...   
4  ['__', 'personales', 'caracter', 'temporal', '...   

                            nombre_del_procedimiento  \
0  apoyo a la gestion como interprete de lengua d...   
1  fortalecer el departamento administrativo de p...   
2  prestacion de servicios de apoyo a la gestion ...   
3                      jaime alberto gomez contreras   
4  instructor area de comercio y ventas - gestion...   

                     nombre_del_procedimiento_limpio estado_contrato  \
0    ['interprete', 'lengua', 'senas', 'colombiana']         cerrado   
1  ['fortalecer', 'departamento', 'administrativo...         cerrado   
2  ['empresa', 'publicos', 'realizacion', 'tratam...      modificado   
3         ['jaime', 'alberto', 'gomez', 'contreras']      modificado   
4  ['instructor', 'area', 'comercio', 'ventas', '...      modificado   

                 departamento         ciudad  ...  \
0                   antioquia         itagui  ...   
1                        meta  villavicencio  ...   
2                   santander         suaita  ...   
3  distrito capital de bogota         bogota  ...   
4          norte de santander         cucuta  ...   

                 nombre_de_la_unidad_de_contratación proveedores_invitados  \
0              contratacion servicios personales ctm                   0.0   
1  departamento administrativo de planeacion depa...                   0.0   
2                                 secretaria general                   0.0   
3                                          contratos                   0.0   
4           centro desarrollo rural y minero- cedrum                   0.0   

  proveedores_con_invitacion_directa visualizaciones_del_procedimiento  \
0                                0.0                               0.0   
1                                0.0                               0.0   
2                                0.0                               0.0   
3                                0.0                               0.0   
4                                0.0                               0.0   

   numero_de_lotes codigo_principal_de_categoria       tipo_de_contrato_y  \
0              0.0                   v1.80111600  prestacion de servicios   
1              0.0                   v1.80111603  prestacion de servicios   
2              0.0                   v1.80111620      decreto 092 de 2017   
3              0.0                   v1.80111701  prestacion de servicios   
4              0.0                   v1.80111600  prestacion de servicios   

  estado_resumen duracion_proceso_dias sin_modificaciones  
0     adjudicado                 300.0               True  
1     adjudicado                  30.0               True  
2     adjudicado                 240.0              False  
3     adjudicado                 150.0              False  
4     adjudicado             

In [18]:
# Total nulls before
total_nulls_before = df.isnull().sum().sum()
print(f"Total nulls before filling: {total_nulls_before}")

# Fill with median for numeric columns
df = df.fillna(df.median(numeric_only=True))

# Fill with mode for non-numeric columns
for col in df.select_dtypes(exclude='number').columns:
    mode_val = df[col].mode(dropna=True)
    if not mode_val.empty:
        df[col] = df[col].fillna(mode_val[0])

# Total nulls after
total_nulls_after = df.isnull().sum().sum()
print(f"Total nulls after filling: {total_nulls_after}")

Total nulls before filling: 963763
Total nulls after filling: 0


In [19]:
df.head()

id_contrato                                     nombre_entidad  \
0  co1.pccntr.1358665  sena regional antioquia grupo administrativo c...   
1  co1.pccntr.4243635                              departamento del meta   
2  co1.pccntr.4381704  empresa de servicios publicos de acueducto; al...   
3  co1.pccntr.5334124                                                unp   
4  co1.pccntr.2164831  sena regional norte de santander grupo de apoy...   

  nit_entidad_x                                objeto_del_contrato  \
0     899999034  prestacion de los servicios personales de cara...   
1    8920001488  fortalecer el departamento administrativo de p...   
2     901142488  prestacion de servicios de apoyo a la gestion ...   
3     900475780  prestar los servicios de apoyo a la gestion en...   
4     899999034  54_9119_473 prestar servicios personales de ca...   

                          objeto_del_contrato_limpio  \
0  ['personales', 'caracter', 'temporal', 'interp...   
1  ['fortalecer', 'departamento', 'administrativo...   
2  ['empresa', 'publicos', 'realizacion', 'tratam...   
3  ['misionales', 'desarrolla', 'subdireccion', '...   
4  ['__', 'personales', 'caracter', 'temporal', '...   

                            nombre_del_procedimiento  \
0  apoyo a la gestion como interprete de lengua d...   
1  fortalecer el departamento administrativo de p...   
2  prestacion de servicios de apoyo a la gestion ...   
3                      jaime alberto gomez contreras   
4  instructor area de comercio y ventas - gestion...   

                     nombre_del_procedimiento_limpio estado_contrato  \
0    ['interprete', 'lengua', 'senas', 'colombiana']         cerrado   
1  ['fortalecer', 'departamento', 'administrativo...         cerrado   
2  ['empresa', 'publicos', 'realizacion', 'tratam...      modificado   
3         ['jaime', 'alberto', 'gomez', 'contreras']      modificado   
4  ['instructor', 'area', 'comercio', 'ventas', '...      modificado   

                 departamento         ciudad  ...  \
0                   antioquia         itagui  ...   
1                        meta  villavicencio  ...   
2                   santander         suaita  ...   
3  distrito capital de bogota         bogota  ...   
4          norte de santander         cucuta  ...   

                 nombre_de_la_unidad_de_contratación proveedores_invitados  \
0              contratacion servicios personales ctm                   0.0   
1  departamento administrativo de planeacion depa...                   0.0   
2                                 secretaria general                   0.0   
3                                          contratos                   0.0   
4           centro desarrollo rural y minero- cedrum                   0.0   

  proveedores_con_invitacion_directa visualizaciones_del_procedimiento  \
0                                0.0                               0.0   
1                                0.0                               0.0   
2                                0.0                               0.0   
3                                0.0                               0.0   
4                                0.0                               0.0   

   numero_de_lotes codigo_principal_de_categoria       tipo_de_contrato_y  \
0              0.0                   v1.80111600  prestacion de servicios   
1              0.0                   v1.80111603  prestacion de servicios   
2              0.0                   v1.80111620      decreto 092 de 2017   
3              0.0                   v1.80111701  prestacion de servicios   
4              0.0                   v1.80111600  prestacion de servicios   

  estado_resumen duracion_proceso_dias sin_modificaciones  
0     adjudicado                 300.0               True  
1     adjudicado                  30.0               True  
2     adjudicado                 240.0              False  
3     adjudicado                 150.0              False  
4     adjudicado             

In [20]:
before = len(df)
# Get only numeric columns
numeric_cols = df.select_dtypes(include='number').columns

# Loop through each numeric column and filter out top and bottom 0.1%
for col in numeric_cols:
    lower_threshold = df[col].quantile(0.001)   # 0.1st percentile
    upper_threshold = df[col].quantile(0.999)   # 99.9th percentile
    df = df[(df[col] >= lower_threshold) & (df[col] <= upper_threshold)]

after = len(df)
print(f"Removed {before - after} rows with extreme values.")

Removed 52690 rows with extreme values.


In [21]:
df.head()

id_contrato                                     nombre_entidad  \
0  co1.pccntr.1358665  sena regional antioquia grupo administrativo c...   
1  co1.pccntr.4243635                              departamento del meta   
2  co1.pccntr.4381704  empresa de servicios publicos de acueducto; al...   
3  co1.pccntr.5334124                                                unp   
4  co1.pccntr.2164831  sena regional norte de santander grupo de apoy...   

  nit_entidad_x                                objeto_del_contrato  \
0     899999034  prestacion de los servicios personales de cara...   
1    8920001488  fortalecer el departamento administrativo de p...   
2     901142488  prestacion de servicios de apoyo a la gestion ...   
3     900475780  prestar los servicios de apoyo a la gestion en...   
4     899999034  54_9119_473 prestar servicios personales de ca...   

                          objeto_del_contrato_limpio  \
0  ['personales', 'caracter', 'temporal', 'interp...   
1  ['fortalecer', 'departamento', 'administrativo...   
2  ['empresa', 'publicos', 'realizacion', 'tratam...   
3  ['misionales', 'desarrolla', 'subdireccion', '...   
4  ['__', 'personales', 'caracter', 'temporal', '...   

                            nombre_del_procedimiento  \
0  apoyo a la gestion como interprete de lengua d...   
1  fortalecer el departamento administrativo de p...   
2  prestacion de servicios de apoyo a la gestion ...   
3                      jaime alberto gomez contreras   
4  instructor area de comercio y ventas - gestion...   

                     nombre_del_procedimiento_limpio estado_contrato  \
0    ['interprete', 'lengua', 'senas', 'colombiana']         cerrado   
1  ['fortalecer', 'departamento', 'administrativo...         cerrado   
2  ['empresa', 'publicos', 'realizacion', 'tratam...      modificado   
3         ['jaime', 'alberto', 'gomez', 'contreras']      modificado   
4  ['instructor', 'area', 'comercio', 'ventas', '...      modificado   

                 departamento         ciudad  ...  \
0                   antioquia         itagui  ...   
1                        meta  villavicencio  ...   
2                   santander         suaita  ...   
3  distrito capital de bogota         bogota  ...   
4          norte de santander         cucuta  ...   

                 nombre_de_la_unidad_de_contratación proveedores_invitados  \
0              contratacion servicios personales ctm                   0.0   
1  departamento administrativo de planeacion depa...                   0.0   
2                                 secretaria general                   0.0   
3                                          contratos                   0.0   
4           centro desarrollo rural y minero- cedrum                   0.0   

  proveedores_con_invitacion_directa visualizaciones_del_procedimiento  \
0                                0.0                               0.0   
1                                0.0                               0.0   
2                                0.0                               0.0   
3                                0.0                               0.0   
4                                0.0                               0.0   

   numero_de_lotes codigo_principal_de_categoria       tipo_de_contrato_y  \
0              0.0                   v1.80111600  prestacion de servicios   
1              0.0                   v1.80111603  prestacion de servicios   
2              0.0                   v1.80111620      decreto 092 de 2017   
3              0.0                   v1.80111701  prestacion de servicios   
4              0.0                   v1.80111600  prestacion de servicios   

  estado_resumen duracion_proceso_dias sin_modificaciones  
0     adjudicado                 300.0               True  
1     adjudicado                  30.0               True  
2     adjudicado                 240.0              False  
3     adjudicado                 150.0              False  
4     adjudicado             

In [22]:
def log_normalize_dataframe(df):
    df_scaled = df.copy()
    numeric_cols = df_scaled.select_dtypes(include='number').columns

    safe_cols = []
    unsafe_cols = []

    # Split columns into safe (for log1p) and unsafe
    for col in numeric_cols:
        if (df_scaled[col] > -1).all() and df_scaled[col].std() != 0:
            safe_cols.append(col)
        elif df_scaled[col].std() != 0:
            unsafe_cols.append(col)

    # Apply log1p to safe columns
    if safe_cols:
        df_scaled[safe_cols] = np.log1p(df_scaled[safe_cols])
        df_scaled[safe_cols] = MinMaxScaler().fit_transform(df_scaled[safe_cols])

    # Apply linear MinMax scaling to unsafe columns
    if unsafe_cols:
        df_scaled[unsafe_cols] = MinMaxScaler().fit_transform(df_scaled[unsafe_cols])

    return df_scaled

df = log_normalize_dataframe(df)
df.head()

id_contrato                                     nombre_entidad  \
0  co1.pccntr.1358665  sena regional antioquia grupo administrativo c...   
1  co1.pccntr.4243635                              departamento del meta   
2  co1.pccntr.4381704  empresa de servicios publicos de acueducto; al...   
3  co1.pccntr.5334124                                                unp   
4  co1.pccntr.2164831  sena regional norte de santander grupo de apoy...   

  nit_entidad_x                                objeto_del_contrato  \
0     899999034  prestacion de los servicios personales de cara...   
1    8920001488  fortalecer el departamento administrativo de p...   
2     901142488  prestacion de servicios de apoyo a la gestion ...   
3     900475780  prestar los servicios de apoyo a la gestion en...   
4     899999034  54_9119_473 prestar servicios personales de ca...   

                          objeto_del_contrato_limpio  \
0  ['personales', 'caracter', 'temporal', 'interp...   
1  ['fortalecer', 'departamento', 'administrativo...   
2  ['empresa', 'publicos', 'realizacion', 'tratam...   
3  ['misionales', 'desarrolla', 'subdireccion', '...   
4  ['__', 'personales', 'caracter', 'temporal', '...   

                            nombre_del_procedimiento  \
0  apoyo a la gestion como interprete de lengua d...   
1  fortalecer el departamento administrativo de p...   
2  prestacion de servicios de apoyo a la gestion ...   
3                      jaime alberto gomez contreras   
4  instructor area de comercio y ventas - gestion...   

                     nombre_del_procedimiento_limpio estado_contrato  \
0    ['interprete', 'lengua', 'senas', 'colombiana']         cerrado   
1  ['fortalecer', 'departamento', 'administrativo...         cerrado   
2  ['empresa', 'publicos', 'realizacion', 'tratam...      modificado   
3         ['jaime', 'alberto', 'gomez', 'contreras']      modificado   
4  ['instructor', 'area', 'comercio', 'ventas', '...      modificado   

                 departamento         ciudad  ...  \
0                   antioquia         itagui  ...   
1                        meta  villavicencio  ...   
2                   santander         suaita  ...   
3  distrito capital de bogota         bogota  ...   
4          norte de santander         cucuta  ...   

                 nombre_de_la_unidad_de_contratación proveedores_invitados  \
0              contratacion servicios personales ctm                   0.0   
1  departamento administrativo de planeacion depa...                   0.0   
2                                 secretaria general                   0.0   
3                                          contratos                   0.0   
4           centro desarrollo rural y minero- cedrum                   0.0   

  proveedores_con_invitacion_directa visualizaciones_del_procedimiento  \
0                                0.0                               0.0   
1                                0.0                               0.0   
2                                0.0                               0.0   
3                                0.0                               0.0   
4                                0.0                               0.0   

   numero_de_lotes codigo_principal_de_categoria       tipo_de_contrato_y  \
0              0.0                   v1.80111600  prestacion de servicios   
1              0.0                   v1.80111603  prestacion de servicios   
2              0.0                   v1.80111620      decreto 092 de 2017   
3              0.0                   v1.80111701  prestacion de servicios   
4              0.0                   v1.80111600  prestacion de servicios   

  estado_resumen duracion_proceso_dias sin_modificaciones  
0     adjudicado              0.838849               True  
1     adjudicado              0.504738               True  
2     adjudicado              0.806172              False  
3     adjudicado              0.737455              False  
4     adjudicado             

In [23]:
df.to_csv(r'D:\tesis_u_sabana\data\db_1\db_1.csv', sep="~", encoding='utf-8', index=False)

In [24]:
df_tiny = df.sample(n=1000, random_state=7)
df_tiny.to_csv(r'D:\tesis_u_sabana\data\db_0\db_1_tiny.csv', sep="~", encoding='utf-8', index=False)